# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [214]:
# import libraries
import pandas as pd
import sqlalchemy

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import classification_report

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [215]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///Phoebe_database.db')
df = pd.read_sql_table('categorised_messages_v5', engine)

X = df['message']
Y = df.iloc[: , 4:]
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [216]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed_words = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [217]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [218]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [219]:
predictions = pipeline.predict(X_test)

In [220]:
for i in range(len(Y_test.columns)):
    print(Y_test.columns[i], classification_report(Y_test.iloc[:, i], predictions[:, i]))

related              precision    recall  f1-score   support

          0       0.65      0.49      0.56      1213
          1       0.85      0.91      0.88      3993
          2       0.22      0.37      0.27        38

avg / total       0.80      0.81      0.80      5244

request              precision    recall  f1-score   support

          0       0.90      0.98      0.94      4338
          1       0.81      0.47      0.59       906

avg / total       0.88      0.89      0.88      5244

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      5213
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      5244

aid_related              precision    recall  f1-score   support

          0       0.75      0.86      0.80      3069
          1       0.75      0.60      0.67      2175

avg / total       0.75      0.75      0.75      5244

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


search_and_rescue              precision    recall  f1-score   support

          0       0.97      1.00      0.99      5093
          1       0.42      0.03      0.06       151

avg / total       0.96      0.97      0.96      5244

security              precision    recall  f1-score   support

          0       0.98      1.00      0.99      5153
          1       0.25      0.01      0.02        91

avg / total       0.97      0.98      0.97      5244

military              precision    recall  f1-score   support

          0       0.97      1.00      0.98      5078
          1       0.49      0.10      0.17       166

avg / total       0.96      0.97      0.96      5244

child_alone              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5244

avg / total       1.00      1.00      1.00      5244

water              precision    recall  f1-score   support

          0       0.95      1.00      0.97      4919
          1       0.79      0.28

### 6. Improve your model
Use grid search to find better parameters. 

In [221]:
pipeline.get_params() 

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ff5fff9a1e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [211]:
# parameters = {
#     'clf__estimator__n_estimators': [1, 2],
#     'clf__estimator__min_samples_leaf':[1, 2]
# }

# cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# cv.fit(X_train, Y_train)

In [ ]:
# predictions.tuned = cv.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [222]:
filename = 'final_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.